# 🧠 Lightweight solution to a 'Peak' Quantum Circuit with Tensor Networks

This notebook was developed during a quantum hackathon hosted by BlueQubit, a Series A startup focused on making quantum computing accessible across various platforms. Their toolset allows for a unified development environment, abstracting hardware and simulators so teams can focus on circuit logic.

Our objective here is to simulate a 60-qubit quantum circuit where one dominant bitstring occurs with O(1) probability, while all others are exponentially less likely. We use tensor networks to do this efficiently, with help from:

- `Quimb`: constructs and simulates quantum circuits as tensor networks.
- `Cotengra`: finds optimal contraction paths for the tensor network.
- A majority vote mechanism to identify the most likely bitstring through sampling.

Throughout this notebook, I explore technical decisions and implementation challenges, and reflect on why certain methods worked better than expected.

# 📦 Import Required Libraries

We’ll use the following tools:

- `quimb.tensor` to define the circuit and convert it into a tensor network.
- `cotengra` to optimize the contraction path, which becomes critical at scale.
- `numpy`, `collections.defaultdict` for general data handling.
- `multiprocessing` and `time` to help with runtime tracking and performance tuning.

Each of these plays a role in making large quantum simulations practical.

In [1]:
import quimb.tensor as qtn
import numpy as np
import cotengra as ctg
from collections import defaultdict
from multiprocessing import freeze_support
import time

### ⏱️ Format Timing Output

This utility function makes runtime output more human-readable. It’s a minor touch, but helpful for tracking performance at different stages of simulation and optimization.

In [2]:
def format_time(seconds):
    """Return a formatted string for a time duration in minutes and seconds if > 60 sec, otherwise in seconds."""
    if seconds < 60:
        return f"{seconds:.2f} seconds"
    else:
        minutes = int(seconds // 60)
        rem_seconds = seconds % 60
        return f"{minutes} minutes {rem_seconds:.2f} seconds"

### 🗳️ Determine Dominant Bitstring via Majority Vote

The function `compute_majority_vote` aggregates sampled bitstrings to extract the most frequent bit at each position.

> Initially, I was skeptical about this working reliably. But reframing the hidden bitstring as a strong signal amidst weak noise led me to this majority vote logic. It’s simple. We treat each position independently and aggregate statistics across many samples.
> You will need to add some guardrails if looking for string without knowing its value. Either inserting a value as a tie breaker, or not allowing tie breakers to take place and pushing to next iteration to see if it resolves.

In [3]:
def compute_majority_vote(bit_counts, num_qubits):
    """Return the majority vote bitstring from current per-position counts."""
    final_bits = []
    for i in range(num_qubits):
        count_0 = bit_counts[i].get('0', 0)
        count_1 = bit_counts[i].get('1', 0)
        # In case of a tie, choose '1'
        final_bits.append('1' if count_1 > count_0 else '0')
    return ''.join(final_bits)


### ⌛ Begin Benchmarking

We use this timing block to measure the total wall-clock time for our full workflow—from circuit loading to final sampling and bitstring extraction.

In [4]:
overall_start = time.perf_counter()

### 🧠 Load the 60-Qubit Circuit

Here, we initialize the 60-qubit circuit and load it from an OpenQASM file into a tensor network using Quimb.

> You'll likely encounter some warnings from Quimb’s QASM parser. These can be safely ignored in this case, as they don’t affect the resulting tensor network structure.

In [5]:
# Setup: Initialize the circuit with 60 qubits and load the QASM file.
circ = qtn.Circuit(N=60)
print("Loading circuit...")
load_start = time.perf_counter()
tensor_network_circuit = circ.from_openqasm2_file(
    './circuit_3_60q.qasm'
)
load_end = time.perf_counter()
print("Circuit loaded.\n")
print(f"Loading circuit took {format_time(load_end - load_start)}.")

Loading circuit...


/Users/mridul.sarkar/Documents/BlueQubitHackathon/venv/lib/python3.9/site-packages/quimb/tensor/circuit.py:221: SyntaxWarning: Unsupported operation ignored: creg
  warnings.warn(
/Users/mridul.sarkar/Documents/BlueQubitHackathon/venv/lib/python3.9/site-packages/quimb/tensor/circuit.py:221: SyntaxWarning: Unsupported operation ignored: measure
  warnings.warn(


Circuit loaded.

Loading circuit took 0.83 seconds.


### 🧮 Set Up Cotengra Contraction Optimizer

We configure Cotengra's optimizer to find efficient paths for contracting the tensor network.

> This step was surprisingly insightful. I compared different optimizers (`nevergrad`, `cmaes`, `random`) but found `optuna` worked the best. I need to study these optimizer a bit more to study trade off. It takes a while to run so will need to leverage GPUs for that work to make it go faster.

In [2]:
# Setup the contraction optimizer using cotengra.
print("Setting up contraction optimizer...")
opt_start = time.perf_counter()
opt = ctg.ReusableHyperOptimizer(
    parallel=True,
    optlib="optuna",
    max_time="rate:1e8",  # Limit optimization time.
    directory=True,
    progbar=True,
)
opt_end = time.perf_counter()
print("Done setting up contraction optimizer.")

Setting up contraction optimizer...


NameError: name 'time' is not defined

### 🔍 Optimize Sampling Strategy

We rehearse contraction paths for all marginal distributions we’ll need during sampling. This makes actual sampling much faster later on.

> This step felt like prepping a lookup table for all future measurements. Quimb’s design makes it very intuitive to offload this complexity while still having fine-grained control. Definitely something to carry forward when designing future tensor-based simulations.

In [ ]:
# Rehearse the sampling path (pre-optimizes contraction paths for each marginal).
print("Optimizing contraction path...")
path_opt_start = time.perf_counter()
rehs = tensor_network_circuit.sample_gate_by_gate_rehearse(
    group_size=10,
    optimize=opt,
    simplify_sequence="ADCRS"  # Using "ADCRS" for simplification.
)
path_opt_end = time.perf_counter()
print(f"Contraction path optimization took {format_time(path_opt_end - path_opt_start)}.")

### 🎯 Define Target Bitstring

This bitstring is expected to dominate the probability distribution after circuit execution. Including it allows us to verify our majority vote or compare results from simulation to theoretical expectations.

> Optional, but handy for debugging and benchmarking against known expected outcomes.

In [ ]:
# Define target bitstring and number of qubits.
target_bitstring = "110101001011010101111001011100001110101101111010100110110001"
num_qubits = 60

### 📝 Sample and Log Output Bitstrings

Here we perform the actual sampling from the optimized tensor network. Each sample gives us one possible outcome of the circuit measurement. We stream all results to a file for further analysis.  What makes this implementation lightweight a combination of everything we did before, setting `sample_batch_size` to 1, and off loading calculation to determine the strongest bitstring signal. Together all the steps within this system were selected with computational constraint in mind.

> This is the culmination of everything we set up—from QASM parsing to optimizer rehearsal. If you're following the logic of tensor contraction and marginalization, this step shows how it all ties together. The process is repeatable and can be scaled to other circuits with similar structure.

In [ ]:
# Prepare to continuously sample and save each bitstring to a file.
output_file = "./tensor_networks/samples.txt"
with open(output_file, "a") as f:
    print("Starting continuous sampling and appending to file...")
    sample_loop_start = time.perf_counter()
    # Maintain position-wise counts for majority vote.
    position_counts = [defaultdict(int) for _ in range(num_qubits)]
    rng = np.random.default_rng(42)
    sample_batch_size = 1  # One sample per batch.
    sample_count = 0

    # Continuous sampling loop.
    while True:
        sample_iter_start = time.perf_counter()
        # Generate one sample.
        for b in tensor_network_circuit.sample_gate_by_gate(
            sample_batch_size,
            group_size=5,
            optimize=opt,
            simplify_sequence="ADCRS",
            seed=rng
        ):
            sample_iter_end = time.perf_counter()
            sample_time = sample_iter_end - sample_iter_start
            sample_count += 1
            print(f"Sample {sample_count}: {b} (took {format_time(sample_time)})")
            
            # Write sample to file.
            f.write(b + "\n")
            f.flush()  # Ensure persistence in case of interruption.
            
            # Update per-qubit counts.
            for i, bit in enumerate(b):
                position_counts[i][bit] += 1
            
            # Check current majority vote.
            current_vote = compute_majority_vote(position_counts, num_qubits)
            if current_vote == target_bitstring:
                solution_time = time.perf_counter() - sample_loop_start
                print(f"\nTarget bitstring achieved after {sample_count} samples in {format_time(solution_time)}.")

            # Reset timer for next sample.
            sample_iter_start = time.perf_counter()
            
        # Log total sampling time every 10 samples.
        if sample_count % 10 == 0:
            current_time = time.perf_counter()
            elapsed = current_time - sample_loop_start
            print(f"Processed {sample_count} samples in {format_time(elapsed)}.")

overall_end = time.perf_counter()
print(f"Overall process took {format_time(overall_end - overall_start)}.")